# Coastal flood hazard for displacement risk modelling

This tutorial shows how to use the coastal flood hazard return periods maps.

Technical specifications for the flood maps.

**Flood map:**  
Resolution: 1000m x 1000m (approximately)  
Horizontal coordinate System: EPSG:4326 - WGS84.  
Nodata value: None.  

The data is single-banded, where each pixel value denotes the depth of the water in meters.  
*** NOTE: Best way to visualize is to clip the raster values from 0.01m to 100m. This will remove 0 values.

**Representative concentration pathways (RCP) considered for sea level rise:** RCP2.6, RCP4.5, RCP8.5  
**Future years considered for sea level rise:** 2020, 2050, 2100.  
**Return periods considered for flood maps:** 1,10,25,50,100,250,1000.   

The superfolder specifies the name of our DEM, namely venDEM. venDEM_scaled_1km implies that flood computation was done on finer (30m resolution), after which maps were rescaled to 1km (approximately).  
The set of flood maps are partitioned into unit latitude-longitude square tiles inside the superfolder.  
For example, S02E040 represents the latitude-longitude tile with bottom left corner at latitude -2 (2S) and longitude 40 (40E).

In each folder, there is then a combination of RCP and year, and within each RCP_year folder, there is one tif file per return period for the latitude-longitude tile.

In [1]:
import os
import pandas as pd
import numpy as np

#os.chdir('/Users/simonameiler/Documents/WCR/Displacement/global-displacement-risk')
os.chdir('/cluster/project/climate/evelynm/global-displacement-risk') # change back to root folder, not "~/doc"
import coastal_flood_hazard, exposure, vulnerability

Select latitude-longitude tiles for country of interest only.

First, load exposure and get lat/lon max/mins from it.  
Then load the respective flood tiles.

## Load exposure from BEM

In [3]:
from climada.entity.exposures import Exposures

In [130]:
# Load the full dataframe, without further re-aggregation / processing other than adding centroids
gdf_bem_subcomps = exposure.gdf_from_bem_subcomps(cntry_name, opt='full')
gdf_bem_subcomps.head()

,id_1x,iso3,cpx,sector,se_seismo,valhum,valfis,bd_1_floor,bd_2_floor,bd_3_floor,geometry
0,153009269,NPL,3,beds_priv,W1,0.000255,4.889260e-07,0.0,0.0,0.0,POINT (87.55417 27.76250)
1,153009269,NPL,3,beds_priv,UFB3,0.000037,6.990995e-08,0.0,0.0,0.0,POINT (87.55417 27.76250)
2,153009269,NPL,3,beds_priv,UFB2,0.000061,1.164063e-07,0.0,0.0,0.0,POINT (87.55417 27.76250)
3,153009269,NPL,3,beds_priv,INF,0.000009,1.774041e-08,0.0,0.0,0.0,POINT (87.55417 27.76250)
4,153009269,NPL,3,beds_priv,DS3,0.000022,4.259307e-08,0.0,0.0,0.0,POINT (87.55417 27.76250)


In [131]:
# filter and apply impf id
gdf_bem_subcomps = gdf_bem_subcomps[gdf_bem_subcomps['valhum']>1] # filter out rows with basically no population
gdf_bem_subcomps['impf_FL'] = gdf_bem_subcomps.se_seismo.map(vulnerability.DICT_PAGER_FLIMPF_CIMA)

In [ ]:
# replace impf 3 --> 5 for 2-storeys and more
gdf_bem_subcomps.loc[((gdf_bem_subcomps.bd_3_floor+gdf_bem_subcomps.bd_2_floor)>0.5)
                     &(gdf_bem_subcomps.impf_FL==3), "impf_FL"] = 5

In [7]:
# replace impf 4 --> 6 for 2-storeys and more
gdf_bem_subcomps.loc[((gdf_bem_subcomps.bd_3_floor+gdf_bem_subcomps.bd_2_floor)>0.5)
                     &(gdf_bem_subcomps.impf_FL==4), "impf_FL"] = 6

In [48]:
# remove for now unnecessary cols and prepare gdf for CLIMADA Exposure
gdf_bem_subcomps.rename({'valhum' : 'value'}, axis=1)
for col in ['iso3', 'sector', 'valfis', 'se_seismo']:
    gdf_bem_subcomps.pop(col)

In [132]:
gdf_bem_subcomps.valhum.sum()

41350197.53250839

In [133]:
exp = Exposures(gdf_bem_subcomps)
exp.gdf.rename({'valhum': 'value'}, axis=1, inplace=True)
exp.value_unit = 'Pop. count'
exp.gdf['longitude'] = exp.gdf.geometry.x
exp.gdf['latitude'] = exp.gdf.geometry.y
exp.gdf = exp.gdf[~np.isnan(exp.gdf.latitude)]  # drop nan centroids
exp.gdf.head()

,id_1x,iso3,cpx,sector,se_seismo,value,valfis,bd_1_floor,bd_2_floor,bd_3_floor,geometry,impf_FL,longitude,latitude
32,148916083,NPL,3,edu_pub,W1,1.920413,0.003678,0.0,0.0,100.0,POINT (80.95417 29.17083),14,80.954167,29.170833
34,148916083,NPL,3,edu_pub,UFB2,1.169213,0.002239,0.0,0.0,100.0,POINT (80.95417 29.17083),6,80.954167,29.170833
37,148916083,NPL,3,edu_pub,DS1,3.960069,0.007584,0.0,0.0,100.0,POINT (80.95417 29.17083),6,80.954167,29.170833
39,148916083,NPL,3,edu_pub,A,3.961855,0.007587,0.0,0.0,100.0,POINT (80.95417 29.17083),12,80.954167,29.170833
40,148916083,NPL,3,emp_ind,W1,1.181405,0.012255,0.0,0.0,100.0,POINT (80.95417 29.17083),14,80.954167,29.170833


#### Get lat/lon min/max from exposure

In [134]:
lat_min, lat_max, lon_min, lon_max = exp.gdf['latitude'].min(), exp.gdf['latitude'].max(), exp.gdf['longitude'].min(), exp.gdf['longitude'].max()

In [135]:
lat_min, lat_max, lon_min, lon_max

(26.354166672426548, 30.254166672426326, 80.06249999999898, 88.16250000000542)

## Load hazard maps for all tiles covering the lat/lon extent of exposure

In [136]:
from climada.util.constants import SYSTEM_DIR
hazard_dir = SYSTEM_DIR/"hazard"/"coastal_flood"/"venDEM_scaled_1km"

In [137]:
# get tiles covering the exposure extent
tiles = coastal_flood_hazard.find_tiles(lat_min, lat_max, lon_min, lon_max)

In [138]:
tiles

['N26E080',
 'N26E081',
 'N26E082',
 'N26E083',
 'N26E084',
 'N26E085',
 'N26E086',
 'N26E087',
 'N26E088',
 'N27E080',
 'N27E081',
 'N27E082',
 'N27E083',
 'N27E084',
 'N27E085',
 'N27E086',
 'N27E087',
 'N27E088',
 'N28E080',
 'N28E081',
 'N28E082',
 'N28E083',
 'N28E084',
 'N28E085',
 'N28E086',
 'N28E087',
 'N28E088',
 'N29E080',
 'N29E081',
 'N29E082',
 'N29E083',
 'N29E084',
 'N29E085',
 'N29E086',
 'N29E087',
 'N29E088',
 'N30E080',
 'N30E081',
 'N30E082',
 'N30E083',
 'N30E084',
 'N30E085',
 'N30E086',
 'N30E087',
 'N30E088']

In [139]:
CF_haz_2020 = coastal_flood_hazard.generate_hazard_object(tiles, hazard_dir, 'RCP45', '2020', 'FL')

Tile directory /cluster/work/climate/evelynm/climada/data/hazard/coastal_flood/venDEM_scaled_1km/N26E080/RCP45_2020 not found, skipping...
Tile directory /cluster/work/climate/evelynm/climada/data/hazard/coastal_flood/venDEM_scaled_1km/N26E081/RCP45_2020 not found, skipping...
Tile directory /cluster/work/climate/evelynm/climada/data/hazard/coastal_flood/venDEM_scaled_1km/N26E082/RCP45_2020 not found, skipping...
Tile directory /cluster/work/climate/evelynm/climada/data/hazard/coastal_flood/venDEM_scaled_1km/N26E083/RCP45_2020 not found, skipping...
Tile directory /cluster/work/climate/evelynm/climada/data/hazard/coastal_flood/venDEM_scaled_1km/N26E084/RCP45_2020 not found, skipping...
Tile directory /cluster/work/climate/evelynm/climada/data/hazard/coastal_flood/venDEM_scaled_1km/N26E085/RCP45_2020 not found, skipping...
Tile directory /cluster/work/climate/evelynm/climada/data/hazard/coastal_flood/venDEM_scaled_1km/N26E086/RCP45_2020 not found, skipping...
Tile directory /cluster/wor

In [ ]:
CF_haz_2020.intensity

In [ ]:
CF_haz_2020.centroids.plot()

In [ ]:
CF_haz_2020.plot_intensity(event=0)

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

fig, ax = plt.subplots(figsize=(8, 6), subplot_kw={'projection': ccrs.PlateCarree()})
CF_haz_2020.plot_intensity(event=0, axis=ax)

plt.show()


In [ ]:
CF_haz_2020.intensity.max(axis=1).data

In [ ]:
CF_haz_2050 = coastal_flood_hazard.generate_hazard_object(tiles, hazard_dir, 'RCP45', '2050', 'FL')

In [ ]:
CF_haz_2050.plot_intensity(event=0)

In [ ]:
CF_haz_2100 = coastal_flood_hazard.generate_hazard_object(tiles, hazard_dir, 'RCP45', '2100', 'FL')

In [ ]:
CF_haz_2100.plot_intensity(event=0)

In [ ]:
CF_haz_2050.intensity

In [ ]:
CF_haz_2020.frequency.size

In [ ]:
CF_haz_2020.event_id

### Question:
Is it a problem that event_id and event_name are "sanitized" and that intensity crs matrices are stacked vertically during the concat/append routine when calling `generate_hazard_object`?

## Impact functions

### Option 1 - merging damage threshold into impact function

In [ ]:
impf_set_fl = vulnerability.IMPF_SET_FL_IVM

In [ ]:
from climada.entity import ImpactFunc, ImpactFuncSet
impf_set_ivm_step = ImpactFuncSet()

In [ ]:
# The threshold of building damage after which all people are displaced. Below, no-one is displaced.
building_thes = 0.55 # 55% iDMC v1; CIMA: 30% for Somalia to 60% for other countries. 

for imp_id in impf_set_fl.get_ids(haz_type='FL'):
    impf_set_fl.get_func(fun_id=imp_id)
    y = impf_set_fl.get_func(fun_id=imp_id)[0].intensity
    x = impf_set_fl.get_func(fun_id=imp_id)[0].mdd
    flood_thres = np.interp(building_thes, x, y)
    print('ID: '+str(imp_id)+' - threshold stepfunction: '+str(flood_thres))
    impf_set_ivm_step.append(
                ImpactFunc.from_step_impf(
                    intensity=(0,  flood_thres, flood_thres *10),
                    haz_type='FL',
                    impf_id=imp_id,
                    intensity_unit = 'm'
                )
    )

### Impacts

In [ ]:
from climada.engine import ImpactCalc

impcalc = ImpactCalc(exp, impf_set_ivm_step, CF_haz_2020)
impact = impcalc.impact()

In [ ]:
print('Annual average displacement: ' + "{:,.0f}".format(impact.aai_agg))

In [ ]:
freqd_curve = impact.calc_freq_curve()

In [ ]:
freqd_curve.plot()

In [ ]:
impact.plot_hexbin_eai_exposure(pop_name=False, ignore_zero=True)

### Option 2 - postprocessing results by damage threshold

In [ ]:
from climada.engine import ImpactCalc

exp.gdf.rename({'value' : 'valhum'}, axis=1, inplace=True)
exp.gdf['value'] = 1
exp.gdf['impf_FL'] = exp.gdf['se_seismo'].map(vulnerability.DICT_PAGER_FLIMPF_CIMA) 

In [ ]:
# compute damage fractions on buildings; keep "events", i.e. in case of flood RPs individually
impact_bldg = ImpactCalc(exp, vulnerability.IMPF_SET_FL_CIMA, CF_haz_2100).impact(save_mat=True)

In [ ]:
# set displacement-damage threshold(s)
dmg_thresh_low = 0.3
dmg_thresh_med = 0.45
dmg_thresh_high = 0.6

In [ ]:
# decide on whether threshold reached
displ_low = int(impact_bldg.imp_mat > dmg_thresh_low)
displ_med = int(impact_bldg.imp_mat > dmg_thresh_med)
displ_high = int(impact_bldg.imp_mat > dmg_thresh_high)

In [ ]:
# compute displacement impacts (exemplified only for median threshold now); store with gdf. 
# Problem: this likely blows up memory as 0s are now explicit
exp.gdf[['imp_rp1','imp_rp10','imp_rp25','imp_rp100','imp_rp250','imp_rp1000']] = exp.gdf.valhum*displ_med.data

In [ ]:
# explicitly compute AED agg. for median threshold scenario (others analogous)
exp.gdf['aed_med'] = exp.gdf['imp_rp1'] *1 + exp.gdf['imp_rp10'] *1/10 + exp.gdf['imp_rp25']*1/25 + exp.gdf['imp_rp100']*1/100 + exp.gdf['imp_rp250'] *1/250 + exp.gdf['imp_rp1000'] *1/1000

In [ ]:
# aggregate impacts by admin-1
exp.gdf['admin1'] = exposure.assign_admin1_attr(exp.gdf.copy(), exposure.path_admin1_attrs, source='gadm')
gdf_admin1_imps = exp.gdf.groupby('admin1')[['imp_rp1','imp_rp10','imp_rp25','imp_rp100','imp_rp250','imp_rp1000', 'aed_med']].sum()

In [ ]:
# aggregate over whole exposure